# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [71]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy as nltk_accuracy
from sklearn.model_selection import train_test_split


# your code here
df = pd.read_csv(r'C:\Users\User\Desktop\iRonhack\Classes at Iron\Week 20\Day 1\lab-nlp\your-code\twitter_data.csv', encoding='ISO-8859-1')

df.columns = ['target','id','date','flag','user','text']
df


#map
df['is_positive'] = df['target'].map({0: False, 4: True})
df

,target,id,date,flag,user,text,is_positive
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,False
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,False
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,False
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",False
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,False
...,...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,True
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,True
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,True
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,True


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [72]:
import re
import nltk
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
import random


# your code here


df_sampled = df.sample(n=20000, random_state=42)
df_sampled



,target,id,date,flag,user,text,is_positive
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...,False
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...,False
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...,False
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...,False
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick",False
...,...,...,...,...,...,...,...
1374482,4,2051447103,Fri Jun 05 22:02:36 PDT 2009,NO_QUERY,_Jaska,@girlwonder24 Thanks.,True
667014,0,2245469948,Fri Jun 19 16:10:39 PDT 2009,NO_QUERY,julianicolao,"trying to study for the biggest test, next wee...",False
1451234,4,2063022808,Sun Jun 07 01:05:46 PDT 2009,NO_QUERY,ElaineToni,Just finished watching Your Song Presents: Boy...,True
1181412,4,1982082859,Sun May 31 10:29:36 PDT 2009,NO_QUERY,lindseyrd20,@janfran813 awww i can't wait to get one,True


In [73]:
def clean_up(text):
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r"[^a-zA-Z\s']", ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize(text):
    return nltk.word_tokenize(text)

def stem_and_lemmatize(words):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stemmed = [stemmer.stem(word) for word in words]
    lemmatized = [lemmatizer.lemmatize(word) for word in words]
    return lemmatized  

def remove_stopwords(words):
    stop_words = set(stopwords.words('english'))
    return [word for word in words if word.lower() not in stop_words]



df_sampled['clean'] = df_sampled['text'].apply(clean_up)
df_sampled['token'] = df_sampled['clean'].apply(tokenize)
df_sampled['stemmed_and_lemmatized'] = df_sampled['token'].apply(stem_and_lemmatize)
df_sampled['text_processed'] = df_sampled['stemmed_and_lemmatized'].apply(remove_stopwords)

df_sampled



,target,id,date,flag,user,text,is_positive,clean,token,stemmed_and_lemmatized,text_processed
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...,False,Nkluvr eva My poor little dumpling In Holmdel ...,"[Nkluvr, eva, My, poor, little, dumpling, In, ...","[Nkluvr, eva, My, poor, little, dumpling, In, ...","[Nkluvr, eva, poor, little, dumpling, Holmdel,..."
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...,False,I'm off too bed I gotta wake up hella early to...,"[I, 'm, off, too, bed, I, got, ta, wake, up, h...","[I, 'm, off, too, bed, I, got, ta, wake, up, h...","['m, bed, got, ta, wake, hella, early, tomorro..."
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...,False,I havent been able to listen to it yet My spea...,"[I, havent, been, able, to, listen, to, it, ye...","[I, havent, been, able, to, listen, to, it, ye...","[havent, able, listen, yet, speaker, busted]"
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...,False,now remembers why solving a relatively big equ...,"[now, remembers, why, solving, a, relatively, ...","[now, remembers, why, solving, a, relatively, ...","[remembers, solving, relatively, big, equation..."
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick",False,Ate too much feel sick,"[Ate, too, much, feel, sick]","[Ate, too, much, feel, sick]","[Ate, much, feel, sick]"
...,...,...,...,...,...,...,...,...,...,...,...
1374482,4,2051447103,Fri Jun 05 22:02:36 PDT 2009,NO_QUERY,_Jaska,@girlwonder24 Thanks.,True,girlwonder Thanks,"[girlwonder, Thanks]","[girlwonder, Thanks]","[girlwonder, Thanks]"
667014,0,2245469948,Fri Jun 19 16:10:39 PDT 2009,NO_QUERY,julianicolao,"trying to study for the biggest test, next wee...",False,trying to study for the biggest test next week...,"[trying, to, study, for, the, biggest, test, n...","[trying, to, study, for, the, biggest, test, n...","[trying, study, biggest, test, next, week, n't..."
1451234,4,2063022808,Sun Jun 07 01:05:46 PDT 2009,NO_QUERY,ElaineToni,Just finished watching Your Song Presents: Boy...,True,Just finished watching Your Song Presents Boys...,"[Just, finished, watching, Your, Song, Present...","[Just, finished, watching, Your, Song, Present...","[finished, watching, Song, Presents, Boystown]"
1181412,4,1982082859,Sun May 31 10:29:36 PDT 2009,NO_QUERY,lindseyrd20,@janfran813 awww i can't wait to get one,True,janfran awww i can't wait to get one,"[janfran, awww, i, ca, n't, wait, to, get, one]","[janfran, awww, i, ca, n't, wait, to, get, one]","[janfran, awww, ca, n't, wait, get, one]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [74]:
from nltk.probability import FreqDist

all_words = [word for words in df_sampled['text_processed'] for word in words]

freq_dist = FreqDist(all_words)

top_5000_words = freq_dist.most_common(5000)

print("Top 5,000 words example (first 10):", top_5000_words[:100])

top5000 = []

for whatever in top_5000_words:
    for name in whatever:
        top5000.append(name)
        
top5000 = top5000[::2]

top5000



Top 5,000 words example (first 10): [("n't", 2268), ("'s", 2150), ("'m", 1610), ('wa', 1412), ('day', 1217), ('get', 1044), ('like', 945), ('good', 933), ('quot', 921), ('go', 898), ('work', 824), ('today', 772), ('love', 756), ('got', 741), ('time', 729), ('one', 695), ('going', 693), ('u', 683), ('know', 680), ('back', 595), ('want', 571), ('amp', 567), ('really', 550), ('think', 535), ('night', 535), ('im', 532), ('see', 528), ('na', 506), ("'ll", 488), ('ca', 487), ('lol', 483), ('home', 477), ('new', 476), ('still', 475), ('well', 468), ('much', 466), ('ha', 462), ('need', 459), ('feel', 437), ('miss', 435), ('last', 409), ('make', 408), ('tomorrow', 400), ("'re", 375), ('great', 361), ('would', 359), ('morning', 357), ('bad', 337), ('fun', 326), ('sad', 323), ('sleep', 322), ('come', 315), ('wish', 313), ('week', 312), ('tonight', 307), ('say', 305), ('right', 303), ('thing', 298), ('oh', 297), ('friend', 296), ('could', 295), ('nice', 295), ('though', 294), ('haha', 294), ("'", 

["n't",
 "'s",
 "'m",
 'wa',
 'day',
 'get',
 'like',
 'good',
 'quot',
 'go',
 'work',
 'today',
 'love',
 'got',
 'time',
 'one',
 'going',
 'u',
 'know',
 'back',
 'want',
 'amp',
 'really',
 'think',
 'night',
 'im',
 'see',
 'na',
 "'ll",
 'ca',
 'lol',
 'home',
 'new',
 'still',
 'well',
 'much',
 'ha',
 'need',
 'feel',
 'miss',
 'last',
 'make',
 'tomorrow',
 "'re",
 'great',
 'would',
 'morning',
 'bad',
 'fun',
 'sad',
 'sleep',
 'come',
 'wish',
 'week',
 'tonight',
 'say',
 'right',
 'thing',
 'oh',
 'friend',
 'could',
 'nice',
 'though',
 'haha',
 "'",
 'thanks',
 "'ve",
 'wait',
 'gon',
 'bed',
 'look',
 'hope',
 'better',
 'way',
 'lt',
 'getting',
 'hate',
 'twitter',
 'people',
 'hour',
 'sorry',
 'weekend',
 'Thanks',
 'show',
 'little',
 'happy',
 'next',
 'school',
 'Good',
 'doe',
 'Oh',
 'sick',
 'even',
 'take',
 'dont',
 'watching',
 'guy',
 'working',
 'LOL',
 'soon',
 'life',
 'girl',
 'cant',
 'watch',
 'year',
 'x',
 'always',
 'movie',
 'already',
 'everyo

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [75]:

def find_features(document):
    words = set(document)
    features = {word: (word in words) for word in top5000}
    return features


documents = list(zip(df_sampled['text_processed'], df_sampled['is_positive']))


featuresets = [(find_features(doc), category) for (doc, category) in documents]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [76]:


random.seed(42)
random.shuffle(featuresets)
train_set, test_set = train_test_split(featuresets, test_size=0.2)

classifier = NaiveBayesClassifier.train(train_set)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [77]:
# your code here

accuracy = nltk_accuracy(classifier, test_set)
print(f"Accuracy: {accuracy:.4f}")


classifier.show_most_informative_features(10)


Accuracy: 0.7160
Most Informative Features
                     Ugh = True            False : True   =     24.9 : 1.0
                  throat = True            False : True   =     20.1 : 1.0
                    Poor = True            False : True   =     18.7 : 1.0
                   laugh = True             True : False  =     16.0 : 1.0
                 Welcome = True             True : False  =     15.3 : 1.0
                  Follow = True             True : False  =     14.0 : 1.0
                     sad = True            False : True   =     13.7 : 1.0
                horrible = True            False : True   =     12.9 : 1.0
                     ugh = True            False : True   =     12.9 : 1.0
                      Hi = True             True : False  =     12.7 : 1.0
